In [1]:
import os 
import sys 
import logging 
from functools import cache 

logging.basicConfig(level=logging.WARNING)

cur_path = os.path.abspath("../..")
if cur_path not in sys.path: 
    sys.path.append(cur_path)

import numpy as np 
import pandas as pd 
import altair as alt 
from altair import datum
from dotenv import load_dotenv
from subgrounds.subgrounds import Subgrounds, Subgraph
from subgrounds.subgraph import SyntheticField
from subgrounds.pagination import ShallowStrategy

# Required when developing in a jupyter-notebook environment 
load_dotenv('../../../.env')

from utils_notebook.utils import remove_prefix, ddf, load_subgraph
from utils_notebook.vega import output_chart, wide_to_longwide, chart_stack_area_overlay_line_timeseries, stack_order_expr, condition_union
from utils_notebook.queries import QueryManager
from utils_notebook.testing import validate_season_series

In [2]:
sg, bs = load_subgraph()
q = QueryManager(sg, bs)

In [3]:
df = q.query_barn()
df = df.iloc[1:,] # TODO: figure out why this is here 
df.head()

,season,timestamp,reward_fertilized_sprouts,sprouts_rinsable,sprouts,rinsable_percent,fertilizer_total,fertilizer_active,fertilizer_available
2,6075,2022-08-06 17:00:14,0.000000,0.000000,8.644328e+07,0.000000,77000000,14437996.0,62562004.0
3,6076,2022-08-06 18:02:11,2266.788451,2266.788451,8.654976e+07,0.000026,77000000,14469156.0,62530844.0
4,6077,2022-08-06 19:01:04,4557.830445,6824.618896,8.658356e+07,0.000079,77000000,14480159.0,62519841.0
5,6078,2022-08-06 20:00:02,6936.066574,13760.685470,8.659188e+07,0.000159,77000000,14484542.0,62515458.0
6,6079,2022-08-06 21:00:00,9357.109094,23117.794564,8.663729e+07,0.000267,77000000,14500295.0,62499705.0


In [4]:
# sprouts breakdown 
df_sprouts = wide_to_longwide(
    df, 
    "timestamp", 
    ['season', 'timestamp'], 
    ['sprouts_rinsable', 'sprouts', 'rinsable_percent'],
    sort_col="timestamp",
)
colors = {
    'sprouts_rinsable': '#219ebc', 
    'sprouts': '#fb8500', 
    'rinsable_percent': '#023047'
}
chart_sprouts_breakdown = chart_stack_area_overlay_line_timeseries(
    df_sprouts, 
    "timestamp", 
    ['sprouts', 'sprouts_rinsable', 'rinsable_percent'], 
    ['sprouts_rinsable', 'sprouts'], 
    "Sprouts Breakdown",
    separate_y_axes=True, 
    show_exploit_rule=False, 
    xaxis_kwargs=dict(labelExpr="timeFormat(toDate(datum.value), '%b %e, %Y')"),
    yaxis_area_kwargs=dict(title="Sprouts", format=".3~s"), 
    yaxis_line_kwargs=dict(title="Percent", format=".2%"), 
    tooltip_formats={
        'sprouts_rinsable': ',d', 
        'sprouts': ',d', 
        'rinsable_percent': '.2%'
    }, 
    color_map=colors, 
    width=400, 
)
chart_sprouts_breakdown

alt.LayerChart(...)

In [5]:
# fertilizer breakdown 
series_fert = df.iloc[-1,][['fertilizer_available', 'fertilizer_active']]
fert_total = series_fert['fertilizer_available'] + series_fert['fertilizer_active']
df_fert = pd.DataFrame(
    {
        'category': series_fert.index, 
        'value': series_fert.values, 
        'label': [
            f"{c}: {int(v):,} \n({v / fert_total:.1%})" 
            for c, v in zip(series_fert.index, series_fert.values)
        ]
    }
)

chart_fert_breakdown = (
    alt.Chart(df_fert, title="Fertilizer Breakdown")
    .encode(
        theta=alt.Theta("value:Q", stack=True),
        color=alt.Color(
            "category:N", 
            legend=alt.Legend(), 
            scale=alt.Scale(
                domain=['fertilizer_available', 'fertilizer_active'], 
                range=['#7209b7', '#06d6a0']
            )
        ),
        tooltip="label:N"
    )
    .mark_arc(outerRadius=120, innerRadius=75)
)

c = (
    alt.hconcat(chart_sprouts_breakdown, chart_fert_breakdown)
    .resolve_legend(color="independent")
    .resolve_scale(y="independent", color="independent")
    .resolve_axis(y="independent")
)
c

alt.HConcatChart(...)

In [6]:
# TODO: update css for this chart 
output_chart(c)

<IPython.core.display.JSON object>